<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range, todf
from tools.trust_ncg import minimize
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access and implementations of common calculations.

In [3]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.11669551 1.12519412 1.85757473] 


## Adiabatic Flash Calculation
Model mass/energy balance and VLE - all variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [5]:
def model(c):
    Ltot = c.Ftot - c.Vtot  # What's not in the vapor is in the liquid
    V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    L = c.Lx * Ltot
    F = c.Fz * c.Ftot
    mass_balance = (F - V - L) # Mass balance for each component (vectors!)
    mass_balance = (mass_balance/jnp.sum(F))**2

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hl(nL=F, T=c.FT)
    VH = p.Hv(nV=V, T=c.flashT)
    LH = p.Hl(nL=L, T=c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    # Raoults with NRTL activity coefficient correction.  One-liner!
    VLE = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
                - c.Vy*c.flashP)
    VLE = (VLE/jnp.sum(c.Vy*c.flashP))**2

    # Square deviations are weighted by their approx. magnitudes and
    # summed for minimization
    return jnp.sum(mass_balance + energy_balance + VLE)

## What's known (static) and unknown (variable) - automatically combined and tracked.

What's known and unknown can be easily swapped around without having to modify the model.  The DotMap structures can have nested lists of DotMaps with nested lists of arrays, and so forth.

In [6]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
s=DotMap()
s.Ftot=10 # Total Feed moes
s.Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
s.FT = 450 # Feed temperature
s.flashP= 101325 # Flash drum pressure

# Variables (to be solved for with initial guesses and bounds)
v=DotMap()
v.Vy = Comp(s.Fz) # Guess vapor/liquid composition equal to feed
v.Lx = Comp(s.Fz) # Comp - constrains mole fractions to behave like mole fractions!
v.flashT = Range(360, 273.15, s.FT)  # Guess and bounds for flash temperature 
v.Vtot = Range(s.Ftot/2, 0., s.Ftot)  # Guess half of feed in vapor

## Magic happens here
The VSC class combines unknown variables and static parameters into a convenient DotMap structure.  Transformations between known/unknown DotMaps and the flat arrays required by the minimization routine is automated.

The minimization algorithm is a robust Trust-Newton Conjugate Gradient coded to take advantage of JAX automatic Jacobian vector products.  

The initial JAX model compilation is a bit slow, but subsequent repeat calculations are fast.

In [7]:
c=VSC(v,s)
modelf = c.transform(model)
x_min, f = minimize(modelf,c.x, verbosity=1)

0:6, f: 0.06224976735089524
1:5, f: 0.0024304726818986685
2:5, f: 2.226347898210919e-05
3:5, f: 2.944426198412097e-09
Final results:
f: 6.034919165393991e-17
x: [-0.14989822 -0.10167514 -0.41635312  0.23361766  0.16785474 -0.1953328 ]


In [8]:
v_sol = c.xtoc(x_min)
df=todf(v_sol)   # Pandas dataframe to tabulate results
col=list(df.columns)
col[0]=('','value')
df.columns=pd.MultiIndex.from_tuples(col)
df.rename(columns={'vector3':'mole fractions'})

mole fractions                    
          value              1         2         3
FT          450                                   
Ftot         10                                   
Fz                    0.333333  0.333333  0.333333
Lx                    0.311418  0.326803  0.361779
Vtot     3.9739                                   
Vy                    0.366567  0.343236  0.290198
flashP   101325                                   
flashT  352.966